In [ ]:
import pandas as pd
import numpy as np
from dataclasses import dataclass
import os
import math
import shutil
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchinfo import summary as torch_summary
import cv2
import plotly.express as px
from pathlib import Path
from ultralytics import YOLO
import wandb
from wandb.integration.ultralytics import add_wandb_callback


In [ ]:
@dataclass
class Config:
    train_images_folder: str
    train_labels_folder: str
    val_images_folder: str
    val_labels_folder: str
    train_csv: str
    val_csv: str
    yolo_config_yaml: str
    training_output_folder: str
    device: str

    # noinspection PyAttributeOutsideInit
    def init(self, training):
        self.training = training
        if self.training:
            os.makedirs(self.training_output_folder, exist_ok=True)

        self.train_ids = pd.read_csv(self.train_csv)['id'].to_numpy()
        self.val_ids = pd.read_csv(self.val_csv)['id'].to_numpy()

        self.seed = 8675309
        self.batch_size = 32
        self.starting_learning_rate = 3e-4
        self.max_epochs = 40
        self.patience = 4
        self.num_workers = 8 if self.device == 'cuda' else 0
        self.pin_memory = self.num_workers > 0
        self.image_size = 640
        self.use_amp = self.device == 'cuda'
        self.verbose = False

        self.imagenet_mean_cpu_tensor = torch.tensor(imagenet_mean_array)
        self.imagenet_std_cpu_tensor = torch.tensor(imagenet_std_array)
        self.channelwise_imagenet_mean_cpu_tensor = self.imagenet_mean_cpu_tensor.view(3, 1, 1)
        self.channelwise_imagenet_std_cpu_tensor = self.imagenet_std_cpu_tensor.view(3, 1, 1)
        self.imagenet_mean_gpu_tensor = gpu_tensor(imagenet_mean_array)
        self.imagenet_std_gpu_tensor = gpu_tensor(imagenet_std_array)
        self.channelwise_imagenet_mean_gpu_tensor = self.imagenet_mean_gpu_tensor.view(3, 1, 1)
        self.channelwise_imagenet_std_gpu_tensor = self.imagenet_std_gpu_tensor.view(3, 1, 1)

        self.model_name = 'yolo26s.pt'


config: Config = None
""" Set to environment-relevant config before training/inference """;

In [ ]:
local_config = Config(
    train_images_folder='data/license_plates/images/train/',
    train_labels_folder='data/license_plates/labels/train_wrong_format/',
    val_images_folder='data/license_plates/images/val/',
    val_labels_folder='data/license_plates/labels/val_wrong_format/',
    train_csv='data/train.csv',
    val_csv='data/val.csv',
    yolo_config_yaml='data/license_plates.yaml',
    training_output_folder='data_gen/',
    device='cpu',
)
kaggle_config = Config(
    train_images_folder='N/A',
    train_labels_folder='N/A',
    val_images_folder='N/A',
    val_labels_folder='N/A',
    train_csv='N/A',
    val_csv='N/A',
    yolo_config_yaml='/kaggle/input/license_plates_dataset/license_plates.yaml',
    training_output_folder='/kaggle/working/',
    device='cuda',
)

In [ ]:
imagenet_mean_tuple = (0.485, 0.456, 0.406)
imagenet_std_tuple = (0.229, 0.224, 0.225)
imagenet_mean_array = np.array([0.485, 0.456, 0.406], dtype=np.float32)
imagenet_std_array = np.array([0.229, 0.224, 0.225], dtype=np.float32)

def gpu_tensor(numpy_array):
    return torch.tensor(numpy_array, device=config.device)

def gpu_image_tensor_to_numpy_array(image_tensor):
    image = denormalize(image_tensor, config.channelwise_imagenet_mean_gpu_tensor, config.channelwise_imagenet_std_gpu_tensor)
    image = torch.clamp(image, 0, 1)
    image = image.permute(1, 2, 0).cpu().numpy()
    return (image * 255).astype(np.uint8)

def normalize(tensor, mean, std):
    return (tensor - mean) / std

def denormalize(tensor, mean, std):
    return tensor * std + mean

def print_model_torchinfo(model: nn.Module):
    print(torch_summary(model, input_size=(1, 3, config.image_width, config.image_height)))

def print_model(model: nn.Module):
    for name, module in model.named_modules():
        print(name, "->", module.__class__.__name__)

def create_dataloader(dataset, shuffle):
    return DataLoader(dataset, batch_size=config.batch_size, shuffle=shuffle, num_workers=config.num_workers, pin_memory=config.pin_memory, generator=config.generator)

def _num_batches(dataloader):
    return math.ceil(len(dataloader.dataset) / config.batch_size)

In [ ]:
def plot_val_images_with_ground_truth_bounding_boxes(num_images_to_show = 3):
    ids = np.random.choice(config.val_ids, num_images_to_show)
    for image_id in ids:
        image_path = f'{config.val_images_folder}{image_id}.jpg'
        image_label = f'{config.val_labels_folder}{image_id}.txt'
        image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        boxes = []
        with open(image_label, 'r') as label_file:
            for line in label_file:
                coords = [float(c) for c in line.strip().split()[-4:]]
                boxes.append(coords)
        fig = px.imshow(image)
        for (x_min, y_min, x_max, y_max) in boxes:
            fig.add_shape(type='rect', x0=x_min, y0=y_min, x1=x_max, y1=y_max, line_color='orange')
        fig.show()

In [ ]:
# config = local_config
# config.init(training=False)
# plot_val_images_with_ground_truth_bounding_boxes()

In [ ]:
def train_yolo():
    # Initialize wandb run (logs hyperparams and enables automatic metric logging)
    wandb.init(
        project="license-plate-detection",
        job_type="train",
        config={
            "model": config.model_name,
            "epochs": config.max_epochs,
            "batch_size": config.batch_size,
            "image_size": config.image_size,
            "lr0": config.starting_learning_rate,
            "patience": config.patience,
            "seed": config.seed,
            "amp": config.use_amp,
            "device": config.device,
        }
    )

    model = YOLO(config.model_name)

    # Register wandb callback — logs metrics per epoch and checkpoints best model as artifact
    add_wandb_callback(model, enable_model_checkpointing=True)

    # Train — verbose=True enables per-epoch cell output logging
    results = model.train(
        data=config.yolo_config_yaml,
        epochs=config.max_epochs,
        imgsz=config.image_size,
        batch=config.batch_size,
        lr0=config.starting_learning_rate,
        patience=config.patience,
        seed=config.seed,
        amp=config.use_amp,
        workers=config.num_workers,
        device=config.device,
        verbose=True,
        plots=True,
    )

    # Copy best model weights to config.training_output_folder
    best_src = Path(results.save_dir) / "weights" / "best.pt"
    os.makedirs(config.training_output_folder, exist_ok=True)
    best_dst = Path(config.training_output_folder) / "best.pt"
    shutil.copy(str(best_src), str(best_dst))
    print(f"Best model saved to: {best_dst}")

    wandb.finish()
    return results


In [ ]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# wandb_key = user_secrets.get_secret("wandb_key")
# !wandb login $wandb_key

In [ ]:
config = kaggle_config
config.init(training=True)
train_yolo()